In [4]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM


In [2]:
df = pd.read_csv('data_Mazancieux_2018.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28960 entries, 0 to 28959
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Subj_idx       28960 non-null  int64  
 1   Stimulus       28960 non-null  int64  
 2   Response       28960 non-null  int64  
 3   Confidence     28960 non-null  int64  
 4   RT_decision    28960 non-null  float64
 5   RT_confidence  28960 non-null  float64
 6   Task           28960 non-null  object 
dtypes: float64(2), int64(4), object(1)
memory usage: 1.5+ MB


## Run a mixed linear regression model with a random intercept per participant

In [26]:
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.api as sm
import numpy as np

df_copy = df.copy()

df_copy['Confidence_transf'] = df_copy['Confidence'] / 10
#df_copy['Confidence_transf'] = df_copy.groupby(['Subj_idx', 'Task'])['Confidence_transf'].transform(lambda x: (x - x.mean()) / x.std())

# Crear solo el lag 1
df_copy['Confidence_lag_1'] = df_copy.groupby('Subj_idx')['Confidence_transf'].shift(1)

# Filtrar las tareas únicas
tasks = df_copy['Task'].unique()

results = {}

for task in tasks:
    task_df = df_copy[df_copy['Task'] == task].dropna(subset=['Confidence_lag_1'])
    
    y = task_df['Confidence_transf']
    
    X = task_df[['Confidence_lag_1']]
    X = sm.add_constant(X)
    
    model = MixedLM(y, X, groups=task_df['Subj_idx'])
    
    result = model.fit()
    
    results[task] = result.summary()

for task, result in results.items():
    print(f"Resultados para Task = {task} - Modelo mixto con lag 1")
    print(result)
    print("\n")


Resultados para Task = VP - Modelo mixto con lag 1
             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Confidence_transf
No. Observations: 7188    Method:             REML             
No. Groups:       181     Scale:              0.0541           
Min. group size:  39      Log-Likelihood:     12.2044          
Max. group size:  40      Converged:          Yes              
Mean group size:  39.7                                         
----------------------------------------------------------------
                  Coef.  Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------------
const             0.432     0.013  32.197  0.000   0.405   0.458
Confidence_lag_1  0.200     0.012  17.023  0.000   0.177   0.223
Group Var         0.024     0.012                               



Resultados para Task = EF - Modelo mixto con lag 1
             Mixed Linear Model Regression Results
Model:            M